# This Notebook demonstrates a N-gram Programming Model that is built upon an existing N-Gram Sentence Completion Model.

The goal of developing this model is to see how does a simple model perform in comparison to the complex Neural Networks and Deep-Learning Models.

The N-gram Sentence Completion model that was used has been cited.

References: https://www.kaggle.com/code/sauravmaheshkar/auto-completion-using-n-gram-models/notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture

## Importing Packages
import math
import nltk
import random
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## nltk settings
nltk.download('punkt')

# file_path = "drive/MyDrive/N-Gram/tokenizer.json"

# with open(file_path, "r") as f:
#   data = f.read()
 

The test dataset that has been used can be downloaded from here: 

Dataset: https://drive.google.com/file/d/1KaPCSWHMyedKq4F-90ZwpJTxmp9B4hDM/view?usp=sharing

In [ ]:
data = pd.read_csv("drive/MyDrive/N-Gram/test.csv")
data_str = '<EOS>'.join(data['sample'].tolist())


In [ ]:
data_str[:1000]

'n = int ( input ( ) ) \n s = input ( ) \n ans = 0 \n ss = s [ 0 ] \n for i in range ( 1 , n ) : \n \t if ss ! = s [ i ] : \n \t \t ans + = 1 \n \t \t ss = s [ i ] \n print ( ans + 1 )<EOS>n = int ( input ( ) ) \n a = list ( map ( int , input ( ) . split ( ) ) ) \n \n p = 1 0 0 0 \n s = 0 \n \n for i , j in zip ( a , a [ 1 : ] ) : \n if i < j : \n d , m = divmod ( p , i ) \n s + = d \n p = m \n elif i > j : \n p + = s * i \n s = 0 \n \n if s ! = 0 : \n p + = s * a [ - 1 ] \n \n print ( p )<EOS>a = int ( input ( ) ) \n b = int ( input ( ) ) \n c = int ( input ( ) ) \n v = int ( input ( ) ) \n \n ans = 0 \n for i in range ( a + 1 ) : \n for j in range ( b + 1 ) : \n for k in range ( c + 1 ) : \n if 5 0 0 * i + 1 0 0 * j + 5 0 * k = = v : \n ans + = 1 \n \n print ( ans ) \n<EOS>def framod ( n , mod , a = 1 ) : \n for i in range ( 1 , n + 1 ) : \n a = a * i % mod \n return a \n \n def power ( n , r , mod ) : \n if r = = 0 : return 1 \n if r % 2 = = 0 : \n return power ( n * n % mod , r / /

In [ ]:
def preprocess_pipeline(data) -> 'list':

    # Split by newline character

### WHY SPLIT DATA INTO SENTENCES?

    sentences = data.split('<EOS>')
    
    # Remove leading and trailing spaces
    sentences = [s.strip() for s in sentences]
    
    # Drop Empty Sentences
    sentences = [s for s in sentences if len(s) > 0]
    
    # Empty List to hold Tokenized Sentences
    tokenized = []
    
    # Iterate through sentences
    for sentence in sentences:
        
        # Convert to lowercase
        sentence = sentence.lower()
        
        # Convert to a list of words
        token = nltk.word_tokenize(sentence)
        
        # Append to list
        tokenized.append(token)
        
    return tokenized


## Pass our data to this function    
tokenized_sentences = preprocess_pipeline(data_str)

In [ ]:
tokenized_sentences[:1]

In [ ]:
train, test = train_test_split(tokenized_sentences, test_size=0.2, random_state=42)

train, val = train_test_split(train, test_size=0.25, random_state=42)

In [ ]:
def counnt_the_words(sentences) -> 'dict':
  word_counts = {}

  for sentence in sentences:
    for token in sentence:
      if token not in word_counts.keys():
        word_counts[token] = 1
      else:
        word_counts[token] += 1
  return word_counts

In [ ]:
def handling_oov(tokenized_sentences, count_threshold):

  closed_vocabulary = []

  words_count = counnt_the_words(tokenized_sentences)

  for word, count in words_count.items():
    if count >= count_threshold:
      closed_vocabulary.append(word)
  
  return closed_vocabulary

In [ ]:
def unk_tokens(tokenized_sentences, vocabulary, unknown_token = "<unk>"):
  vocabulary = set(vocabulary)

  new_tokenized_sentences = []

  for sentence in tokenized_sentences:
    new_sentence = []
    for token in sentence:
      if token in vocabulary:
        new_sentence.append(token)
      else:
        new_sentence.append(unknown_token)
  
    new_tokenized_sentences.append(new_sentence)

  return new_tokenized_sentences

In [ ]:
def cleansing(train_data, test_data, count_threshold):

  vocabulary = handling_oov(train_data, count_threshold)

  new_train_data = unk_tokens(train_data, vocabulary)

  new_test_data = unk_tokens(test_data, vocabulary)

  return new_train_data, new_test_data, vocabulary

In [ ]:
min_freq = 6
final_train, final_test, vocabulary = cleansing(train, test, min_freq)

In [ ]:
final_train[:5]

In [ ]:
def count_n_grams(data, n, start_token = "<s>", end_token = "<e>") -> 'dict':

  # Empty dict for n-grams
  n_grams = {}
 
  # Iterate over all sentences in the dataset
  for sentence in data:
        
    # Append n start tokens and a single end token to the sentence
    
### WHY MULTIPLY START_TOKENS BY N?

    sentence = [start_token]*n + sentence + [end_token]
    
    # Convert the sentence into a tuple
    sentence = tuple(sentence)

    # Temp var to store length from start of n-gram to end
    m = len(sentence) if n==1 else len(sentence)-1
    
    # Iterate over this length
    for i in range(m):
        
      # Get the n-gram
      n_gram = sentence[i:i+n]
    
      # Add the count of n-gram as value to our dictionary
      # IF n-gram is already present
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      # Add n-gram count
      else:
        n_grams[n_gram] = 1
        
  return n_grams

In [ ]:
def prob_for_single_word(word, previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary_size, k) -> 'float':

  # Convert the previous_n_gram into a tuple 
  previous_n_gram = tuple(previous_n_gram)
    
  # Calculating the count, if exists from our freq dictionary otherwise zero
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
  
  # The Denominator
  denom = previous_n_gram_count + k * vocabulary_size

  # previous n-gram plus the current word as a tuple
  nplus1_gram = previous_n_gram + (word,)

  # Calculating the nplus1 count, if exists from our freq dictionary otherwise zero 
  nplus1_gram_count = nplus1_gram_counts[nplus1_gram] if nplus1_gram in nplus1_gram_counts else 0

  # Numerator
  num = nplus1_gram_count + k

  # Final Fraction
  prob = num / denom
  return prob

In [ ]:
def probs(previous_n_gram, n_gram_counts, nplus1_gram_counts, vocabulary, k) -> 'dict':

  # Convert to Tuple
  previous_n_gram = tuple(previous_n_gram)

  # Add end and unknown tokens to the vocabulary

# WHY NOT ADD START TOKENS TOO?

  # Calculate the size of the vocabulary
  vocabulary_size = len(vocabulary)

  # Empty dict for probabilites
  probabilities = {}

  # Iterate over words 
  for word in vocabulary:
    
    # Calculate probability
    probability = prob_for_single_word(word, previous_n_gram, 
                                           n_gram_counts, nplus1_gram_counts, 
                                           vocabulary_size, k=k)
    # Create mapping: word -> probability
    probabilities[word] = probability

  return probabilities

In [ ]:
def auto_complete(previous_tokens, n_gram_counts, nplus1_gram_counts, vocabulary, k, num_suggestions, start_with=None):

    # length of previous words
    n = len(list(n_gram_counts.keys())[0]) 

    # most recent 'n' words
    previous_n_gram = previous_tokens[-n:]
    
    # Calculate probabilty for all words
    probabilities = probs(previous_n_gram,n_gram_counts, nplus1_gram_counts,vocabulary, k=k)

    # Intialize the suggestion and max probability
    suggestion = []
    max_prob = 0

    # Iterate over all words and probabilites, returning the max.
    # We also add a check if the start_with parameter is provided
    for word, prob in probabilities.items():
        
        # if start_with != None: 
            
        #     if not word.startswith(start_with):
        #         continue 

        if prob > max_prob: 
          if len(suggestion) < num_suggestions:
            suggestion.append((word, prob))
          else:
            flag = False
            idx = 0
            while flag == False and idx < len(suggestion):
              if suggestion[idx][1] < prob:
                flag = True
                suggestion[idx] = (word, prob)
              idx += 1

    return suggestion

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k, start_with=None, num_suggestions=5):

    # See how many models we have
    count = len(n_gram_counts_list)
    
    # Empty list for suggestions
    suggestions = []
    
    # IMP: Earlier "-1"
    
    # Loop over counts
    for i in range(count-1):
        
        # get n and nplus1 counts
        n_gram_counts = n_gram_counts_list[i]
        nplus1_gram_counts = n_gram_counts_list[i+1]
        
        # get suggestions 
        suggestion = auto_complete(previous_tokens, n_gram_counts,
                                    nplus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with, num_suggestions=num_suggestions)
        # Append to list
        suggestions.append(suggestion)
        
    return suggestions

In [ ]:
n = 3
n_gram_counts_list = []
for n in range(n, n+2):
    n_model_counts = count_n_grams(final_train, n)
    n_gram_counts_list.append(n_model_counts)

In [ ]:
previous_tokens = ["n", "=", "int", "("]
suggestion = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=2.0, num_suggestions=10)

display(suggestion)


[[('s', 0.013278085202149912),
  ('next', 0.009055045142842014),
  ('a', 0.014257724588948608),
  ('x', 0.0058778363207921845),
  ('raw', 0.005666022399322196),
  ('(', 0.005388016627392835),
  ('math', 0.003044825121131086),
  ('input', 0.5852286266514867),
  (')', 0.0025417670576398633),
  ('c', 0.0024623368370886175)]]